# Test Dataset Samples

In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.image as mpl_img

from htc.context.settings_context import settings_context
from htc.models.image.DatasetImage import DatasetImage
from htc.settings_seg import settings_seg
from htc.tivita.DataPath import DataPath
from htc.utils.Config import Config

In [2]:
# Alternatives for occlusion: P062#2020_05_15_18_58_07, P062#2020_05_15_19_58_26
for image_name, file_name in [
    ("P041#2019_12_14_12_29_18", "real"),
    ("P046#2020_02_07_09_10_07", "no-occlusion"),
    ("P062#2020_05_15_20_00_49", "occlusion"),
]:
    path = DataPath.from_image_name(image_name)
    mpl_img.imsave(settings_context.paper_dir / f"sample_bladder_{file_name}.png", path.read_rgb_reconstructed())

In [3]:
path = DataPath.from_image_name("P046#2020_02_07_09_14_23")
mpl_img.imsave(settings_context.paper_dir / "sample_bladder_raw.png", path.read_rgb_reconstructed())

In [4]:
mapping = settings_seg.label_mapping
config = Config({"label_mapping": mapping, "input/n_channels": 3})
sample_original = DatasetImage([path], train=False, config=config)[0]
assert mapping.name_to_index("background") in sample_original["labels"]

for transform_class, target_label, transform_name in [
    ("OrganIsolation", 4, "isolation"),
    ("OrganRemoval", 5, "removal"),
]:
    for fill_value, fill_name in [("0", "zero"), ("cloth", "cloth")]:
        config_ttt = Config(
            {
                "label_mapping": mapping,
                "input/n_channels": 3,
                "input/test_time_transforms_cpu": [
                    {
                        "class": f"htc.context.context_transforms>{transform_class}",
                        "fill_value": fill_value,
                        "target_label": target_label,
                    }
                ],
            }
        )
        dataset = DatasetImage([path], train=False, config=config_ttt)
        sample_transformed = dataset[0]
        mpl_img.imsave(
            settings_context.paper_dir / f"sample_bladder_{transform_name}_{fill_name}.png",
            sample_transformed["features"].numpy(),
        )